In [1]:
# Huge thanks to the EIA for providing this API
# See https://www.eia.gov/opendata/documentation.php for detailed API documentation

from eia import Client

In [2]:
client = Client()

In [3]:
# Dataset Info includes all non-leaf-node data series (i.e. parents of other datasets or of series)
electricity = client.dataset_info("electricity")
# Routes are child datasets or series inside a dataset
electricity.routes

[RouteInfo(id='retail-sales', name='Electricity Sales to Ultimate Customers', description='Electricity sales to ultimate customer by state and sector (number of customers, average price, revenue, and megawatthours of sales).  \n    Sources: Forms EIA-826, EIA-861, EIA-861M'),
 RouteInfo(id='electric-power-operational-data', name='Electric Power Operations (Annual and Monthly)', description='Monthly and annual electric power operations by state, sector, and energy source.\n    Source: Form EIA-923'),
 RouteInfo(id='rto', name='Electric Power Operations (Daily and Hourly)', description='Hourly and daily electric power operations by balancing authority.  \n    Source: Form EIA-930'),
 RouteInfo(id='state-electricity-profiles', name='State Specific Data', description='State Specific Data'),
 RouteInfo(id='operating-generator-capacity', name='Inventory of Operable Generators', description='Inventory of operable generators in the U.S.\n    Source: Forms EIA-860, EIA-860M'),
 RouteInfo(id='fa

In [4]:
# Series Info includes data series with specific data elements and facets
operational = client.series_info("electricity/electric-power-operational-data")
# facet_options includes information on specific values for facets
operational.facet_options

{'location': [FacetOption(id='NC', name='North Carolina', alias='(NC) North Carolina'),
  FacetOption(id='HI', name='Hawaii', alias='(HI) Hawaii'),
  FacetOption(id='ENC', name='East North Central', alias='Region: (ENC) East North Central'),
  FacetOption(id='MA', name='Massachusetts', alias='(MA) Massachusetts'),
  FacetOption(id='WNC', name='West North Central', alias='Region: (WNC) West North Central'),
  FacetOption(id='VT', name='Vermont', alias='(VT) Vermont'),
  FacetOption(id='WA', name='Washington', alias='(WA) Washington'),
  FacetOption(id='MTN', name='Mountain', alias='Region: (MTN) Mountain'),
  FacetOption(id='MN', name='Minnesota', alias='(MN) Minnesota'),
  FacetOption(id='US', name='U.S. Total', alias='Total: (US) U.S. Total'),
  FacetOption(id='SAT', name='South Atlantic', alias='Region: (SAT) South Atlantic'),
  FacetOption(id='MD', name='Maryland', alias='(MD) Maryland'),
  FacetOption(id='VA', name='Virginia', alias='(VA) Virginia'),
  FacetOption(id='DC', name='Di

In [5]:
# Set get_facet_info to False to avoid retrieving available *values* for facets
# The available facets are still returned in the original info request
info = client.series_info("petroleum/pri/gnd", get_facet_info=False)
# The data attribute gives available data fields for the series
info.data

{'value': DataInfo(alias=None, units=None)}

In [6]:
# Info responses are cached, so the second call is much faster
operational = client.series_info("electricity/electric-power-operational-data")
operational.facets

[FacetInfo(id='location', description='State / Census Region'),
 FacetInfo(id='sectorid', description='Sector'),
 FacetInfo(id='fueltypeid', description='Energy Source')]

In [7]:
# Getting data returns a dataframe object
df = client.get(
    "electricity/electric-power-operational-data", 
    data=["generation", "total-consumption"], 
    facets={"fueltypeid": ["SUB"]}
)
df.head()

,period,location,stateDescription,sectorid,sectorDescription,fueltypeid,fuelTypeDescription,generation,generation-units,total-consumption,total-consumption-units
0,2022-12,OH,Ohio,99,All Sectors,SUB,subbituminous coal,38.14917,thousand megawatthours,16.448,thousand short tons
1,2022-11,MO,Missouri,1,Electric Utility,SUB,subbituminous coal,2392.41242,thousand megawatthours,1409.334,thousand short tons
2,2022-11,KY,Kentucky,98,Electric Power,SUB,subbituminous coal,729.52632,thousand megawatthours,449.992,thousand short tons
3,2021-08,ENC,East North Central,1,Electric Utility,SUB,subbituminous coal,2888.14841,thousand megawatthours,1792.265,thousand short tons
4,2021-08,ENC,East North Central,2,IPP Non-CHP,SUB,subbituminous coal,1923.01959,thousand megawatthours,1225.726,thousand short tons


In [8]:
# We iterate over available records to get all data, using multiple requests if necessary
len(df)

74173

In [9]:
df = client.get(
    "electricity/electric-power-operational-data", 
    data=["generation", "total-consumption"], 
    facets={"fueltypeid": ["SUB"], "location": ["NC"], "sectorid": ["99"]}
)
df.head()

,period,location,stateDescription,sectorid,sectorDescription,fueltypeid,fuelTypeDescription,generation,generation-units,total-consumption,total-consumption-units
0,2005-05,NC,North Carolina,99,All Sectors,SUB,subbituminous coal,None,thousand megawatthours,None,thousand short tons
1,2005-06,NC,North Carolina,99,All Sectors,SUB,subbituminous coal,None,thousand megawatthours,None,thousand short tons
